In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime as dt
import matplotlib.pyplot as plt
from matplotlib import style
import pandas as pd
import pandas_datareader.data as web
from matplotlib.finance import candlestick_ohlc
import matplotlib.dates as mdates
import talib

def get_data(ticker):
    start = dt.datetime(2015, 1, 1)
    end = dt.datetime(2017, 12, 31)
    df = web.DataReader('ticker', 'yahoo', start, end)
    df.to_csv('{ticker}.csv')
    
get_data('AMZN')

In [ ]:
def data_preprocessing(ticker):
    dataset = pd.read_csv('{}.csv'.format(ticker))
    dataset = dataset.dropna()
    dataset = dataset[['Open', 'High', 'Low', 'Close']]
    
    dataset['H-L'] = dataset['High'] - dataset['Low']
    dataset['O-C'] = dataset['Close'] - dataset['Open']
    
    dataset['ma_5'] = dataset['Close'].rolling(window=5).mean()
    dataset['ma_10'] = dataset['Close'].rolling(window=10).mean()

    dataset['std_5'] = dataset['Close'].rolling(window=5).std()
    dataset['std_10'] = dataset['Close'].rolling(window=10).std()
    
    dataset['RSI'] = talib.RSI(dataset['Close'].values, timeperiod = 14)
    dataset['Williams %R'] = talib.WILLR(dataset['High'].values, dataset['Low'].values, dataset['Close'].values, 7)
    dataset['SAR'] = talib.SAR(np.array(dataset['High']), np.array(dataset['Low']), 0.2, 0.2)
    dataset['ADX'] = talib.ADX(np.array(dataset['High']), np.array(dataset['Low']), np.array(dataste['Close']), 10)

    dataset['Price_Rise'] = np.where(dataset['Close'].shift(-1) > dataset['Close'], 1, 0)
    
    dataset = dataset.dropna()
    X = dataset.iloc[:, 4:-1]
    y = dataset.iloc[:, -1]
    
    split = int(len(dataset)*0.8)
    X_train, X_test, y_train, y_test = X[:split], X[split:], y[:split], y[split:]
    
    return X_train, X_test, y_train, y_test

data_preprocessing('AMZN')

In [ ]:
from sklearn import svm, cross_validation, neighbors
from collections import Counter

def svm_linear(ticker):
    X_train, X_test, y_train, y_test = data_preprocessing(ticker)
    clf = svm.SVC(kernel = 'linear')
    clf.fit(X_train, y_train)
    confidence = clf.score(X_test, y_test)
    print('accuracy:',confidence)
    predictions = clf.predict(X_test)
    print('predicted class counts:',Counter(predictions))
    
svm_linear('AMZN')    

In [ ]:
def svm_poly(ticker):
    X_train, X_test, y_train, y_test = data_preprocessing(ticker)
    clf = svm.SVC(kernel = 'poly')
    clf.fit(X_train, y_train)
    confidence = clf.score(X_test, y_test)
    print('accuracy:',confidence)
    predictions = clf.predict(X_test)
    print('predicted class counts:',Counter(predictions))
    
svm_poly('AMZN')   

In [ ]:
def svm_rbf(ticker):
    X_train, X_test, y_train, y_test = data_preprocessing(ticker)
    clf = svm.SVC(kernel = 'rbf')
    clf.fit(X_train, y_train)
    confidence = clf.score(X_test, y_test)
    print('accuracy:',confidence)
    predictions = clf.predict(X_test)
    print('predicted class counts:',Counter(predictions))
    
svm_rbf('AMZN')  

In [ ]:
def KNN(ticker):
    X_train, X_test, y_train, y_test = data_preprocessing(ticker)
    
    clf = neighbors.KNeighborsClassifier()
    clf.fit(X_train, y_train)
    confidence = clf.score(X_test, y_test)
    print('accuracy:',confidence)
    predictions = clf.predict(X_test)
    print('predicted class counts:',Counter(predictions))
    
KNN('AMZN') 

In [ ]:
from sklearn.ensemble import VotingClassifier, RandomForestClassifier

def voting_classifier(ticker):
    X_train, X_test, y_train, y_test = data_preprocessing(ticker)

    clf = VotingClassifier([('lsvc',svm.LinearSVC()),
                            ('knn',neighbors.KNeighborsClassifier()),
                            ('rfor',RandomForestClassifier())])

    clf.fit(X_train, y_train)
    confidence = clf.score(X_test, y_test)
    print('accuracy:',confidence)
    predictions = clf.predict(X_test)
    print('predicted class counts:',Counter(predictions))
    print()
    print()
    return confidence

voting_classifier('AMZN')